In [4]:
#!/usr/bin/env python3
"""
Compute Endurosat X-band link margin vs. elevation for two altitudes and
export both a PNG plot and a CSV table.

Inputs pulled from CSV (Endurosat X-band column):
- selected_carrier_frequency [GHz]
- tx_power [W]
- peak_boresight_antenna_gain [dBi]
- interconnect_losses_gains [dB]  (positive = loss)
- atmospheric_attenuation, rain_attenuation, scintillation_loss,
  polarisation_loss, other_propagation_losses [dB]
- g_t [dB/K]
- data_rate_in_db_hz [dB-Hz]
- codmod_inferred_required_eb_n0 [dB]
- other_overall_link_losses, gain_rolloff_loss_at_pointing_error [dB]

Outputs:
- ./Endurosat_X-band_link_margin_vs_elevation.csv
- ./Endurosat_X-band_link_margin_vs_elevation.png
"""

import os
import math
import argparse
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

EARTH_RADIUS_KM = 6371.0
TARGET_COLUMN = "Endurosat X-band"

# -------- helpers --------
def to_float(x):
    try:
        return float(str(x).replace("%", "").strip())
    except Exception:
        return None

def resolve_csv_path(cli_csv: str | None):
    if cli_csv and cli_csv.lower().endswith(".csv") and Path(cli_csv).is_file():
        return cli_csv
    for p in ["./x_down_info.csv",
              os.path.expanduser("~/Downloads/x_down_info.csv"),
              "/mnt/data/x_down_info.csv"]:
        if Path(p).is_file():
            return p
    raise FileNotFoundError(
        "CSV not found. Pass a path explicitly (e.g. --csv C:\\path\\x_down_info.csv) "
        "or place x_down_info.csv in the current folder."
    )

def get_var(df: pd.DataFrame, varname: str, default=None):
    s = df.loc[df["python_variable"] == varname, TARGET_COLUMN]
    return to_float(s.iloc[0]) if not s.empty else default

def slant_range_km(alt_km: float, elev_deg: float) -> float:
    e = math.radians(elev_deg)
    R = EARTH_RADIUS_KM
    H = R + alt_km
    return -R * math.sin(e) + math.sqrt((R * math.sin(e)) ** 2 + H ** 2 - R ** 2)

def fspl_db(range_km: float, freq_ghz: float) -> float:
    # Standard: 92.45 + 20log10(R[km]) + 20log10(f[GHz])
    return 92.45 + 20 * math.log10(range_km) + 20 * math.log10(freq_ghz)

def eirp_dbw(P_w: float, G_tx_dbi: float, L_tx_db: float) -> float:
    return 10 * math.log10(P_w) + (G_tx_dbi or 0.0) - (L_tx_db or 0.0)

def cn0_dbhz(eirp_dbw: float, L_total_db: float, G_T_dbk: float,
             k_val: float = 1.38e-23, extra_losses_db: float = 0.0) -> float:
    k_db = 10 * math.log10(k_val)
    return eirp_dbw - L_total_db + (G_T_dbk or 0.0) - k_db - (extra_losses_db or 0.0)

# -------- main compute --------
def compute_table(csv_path: str,
                  altitudes=(500, 600),
                  elevations=(0, 5, 10, 15, 20)) -> pd.DataFrame:
    df = pd.read_csv(csv_path)

    # constants from CSV
    f_ghz = get_var(df, "selected_carrier_frequency")
    tx_power_w = get_var(df, "tx_power")
    Gtx_dbi = get_var(df, "peak_boresight_antenna_gain")
    Ltx_db = get_var(df, "interconnect_losses_gains", 0.0)

    atm = get_var(df, "atmospheric_attenuation", 0.0)
    rain = get_var(df, "rain_attenuation", 0.0)
    scint = get_var(df, "scintillation_loss", 0.0)
    pol = get_var(df, "polarisation_loss", 0.0)
    other_prop = get_var(df, "other_propagation_losses", 0.0)

    G_T = get_var(df, "g_t")
    Rb_dBHz = get_var(df, "data_rate_in_db_hz")
    EbN0_req = get_var(df, "codmod_inferred_required_eb_n0", 0.0)
    other_overall = get_var(df, "other_overall_link_losses", 0.0)
    rolloff = get_var(df, "gain_rolloff_loss_at_pointing_error", 0.0)

    rows = []
    for alt in altitudes:
        for el in elevations:
            sr_km = slant_range_km(alt, el)
            Lfs = fspl_db(sr_km, f_ghz)
            Ltotal = Lfs + atm + rain + scint + pol + other_prop
            eirp = eirp_dbw(tx_power_w, Gtx_dbi, Ltx_db)
            cn0 = cn0_dbhz(eirp, Ltotal, G_T, 1.38e-23, other_overall + rolloff)
            ebn0 = cn0 - Rb_dBHz
            margin = ebn0 - EbN0_req

            rows.append({
                "orbit_altitude_km": alt,
                "elevation_deg": el,
                "slant_range_km": sr_km,
                "fspl_db": Lfs,
                "total_prop_loss_db": Ltotal,
                "eirp_dbw": eirp,
                "cn0_dbhz": cn0,
                "ebn0_db": ebn0,
                "link_margin_db": margin,
            })

    return pd.DataFrame(rows)

def make_plot(df_out: pd.DataFrame, png_path: str):
    plt.figure(figsize=(8, 6))
    for alt in sorted(df_out["orbit_altitude_km"].unique()):
        subset = df_out[df_out["orbit_altitude_km"] == alt].sort_values("elevation_deg")
        plt.plot(subset["elevation_deg"], subset["link_margin_db"], marker="o", label=f"{int(alt)} km")
    plt.title("Link Margin vs Elevation for Endurosat X-band")
    plt.xlabel("Elevation (degrees)")
    plt.ylabel("Link Margin (dB)")
    plt.xticks(sorted(df_out["elevation_deg"].unique()))
    plt.grid(True)
    plt.legend()
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()

def main():
    parser = argparse.ArgumentParser(description="Generate Link Margin vs Elevation plot and CSV.")
    parser.add_argument("--csv", help="Path to x_down_info.csv")
    parser.add_argument("--out-csv", default="Endurosat_X-band_link_margin_vs_elevation.csv")
    parser.add_argument("--out-png", default="Endurosat_X-band_link_margin_vs_elevation.png")
    parser.add_argument("--alts", nargs="*", type=float, default=[500, 600],
                        help="Altitudes in km (space-separated)")
    parser.add_argument("--elevs", nargs="*", type=float, default=[0, 5, 10, 15, 20],
                        help="Elevations in degrees (space-separated)")

    args, _ = parser.parse_known_args()
    csv_path = resolve_csv_path(args.csv)

    df_out = compute_table(csv_path, altitudes=args.alts, elevations=args.elevs)
    df_out.to_csv(args.out_csv, index=False)
    make_plot(df_out, args.out_png)

    print(f"Wrote CSV -> {args.out_csv}")
    print(f"Wrote PNG -> {args.out_png}")

if __name__ == "__main__":
    main()


Wrote CSV -> Endurosat_X-band_link_margin_vs_elevation.csv
Wrote PNG -> Endurosat_X-band_link_margin_vs_elevation.png


In [5]:
#!/usr/bin/env python3
"""
Notebook/CLI friendly pass data budget generator.

- If run with CLI args, behaves like before.
- If run with *no* args (e.g., pasted into a Jupyter cell), it:
  * looks for Endurosat_X-band_link_margin_vs_elevation.csv and x_down_info.csv
    in ./ or /mnt/data
  * finds all altitudes in the link CSV
  * writes pass_data_budget_<alt>km.csv for each altitude
"""

import argparse, math, sys, os
from pathlib import Path
import pandas as pd

MU_EARTH = 398600.4418  # km^3/s^2
R_EARTH = 6371.0        # km

def find_file(name):
    for p in [Path(name), Path("/mnt/data")/name]:
        if p.is_file():
            return str(p)
    return None

def get_supported_rate(info_csv: str):
    df = pd.read_csv(info_csv)
    s = df.loc[df["python_variable"].astype(str)=="supported_user_data_rate"]
    if s.empty:
        s = df[df["Parameter"].astype(str).str.contains("Supported user data rate", case=False, na=False)]
    if s.empty:
        raise ValueError("Could not find 'Supported user data rate' in x_down_info.csv")
    val = s.iloc[0]["Endurosat X-band"]
    units = s.iloc[0]["Units"] if "Units" in df.columns else ""
    rate_val = float(str(val).strip())
    unit_str = (units or "").lower()
    if "mb" in unit_str:  bps, pretty = rate_val*1e6, "Mb/s"
    elif "kb" in unit_str: bps, pretty = rate_val*1e3, "kb/s"
    elif "b/s" in unit_str or "bps" in unit_str: bps, pretty = rate_val, "b/s"
    else: bps, pretty = rate_val*1e6, "Mb/s (assumed)"
    return bps, pretty

def central_angle_from_slant(range_km: float, alt_km: float) -> float:
    H = R_EARTH + alt_km
    rho = range_km
    cos_theta = (R_EARTH*R_EARTH + H*H - rho*rho) / (2*R_EARTH*H)
    return math.acos(max(-1.0, min(1.0, cos_theta)))

def mean_motion_rad_s(alt_km: float) -> float:
    a = R_EARTH + alt_km
    return math.sqrt(MU_EARTH / (a*a*a))

def build_budget(link_csv: str, info_csv: str, alt_km: float,
                 emin: float | None, margin_min: float, out_csv: str) -> pd.DataFrame:
    df = pd.read_csv(link_csv)
    need = {"orbit_altitude_km","elevation_deg","link_margin_db","slant_range_km"}
    if not need.issubset(df.columns):
        raise ValueError(f"link CSV missing columns: {need - set(df.columns)}")
    dfa = df[df["orbit_altitude_km"].astype(float)==float(alt_km)].copy()
    if dfa.empty:
        raise ValueError(f"No rows for orbit_altitude_km={alt_km}")
    dfa.sort_values("elevation_deg", inplace=True)
    bps, unit_str = get_supported_rate(info_csv)
    if emin is None:
        emin = float(dfa["elevation_deg"].min())

    e = dfa["elevation_deg"].to_list()
    m = dfa["link_margin_db"].to_list()
    r = dfa["slant_range_km"].to_list()
    theta = [central_angle_from_slant(x, alt_km) for x in r]
    n = mean_motion_rad_s(alt_km)

    rows = []
    for i in range(len(e)-1):
        e0, e1 = e[i], e[i+1]
        m0, m1 = m[i], m[i+1]
        th0, th1 = theta[i], theta[i+1]
        if max(e0,e1) < emin: continue
        e_start, e_end = max(min(e0,e1), emin), max(e0,e1)
        if e_end <= e_start: continue

        def lerp(x0,y0,x1,y1,x):
            t=(x-x0)/(x1-x0); return y0+t*(y1-y0)

        if e1 < e0:  # ensure ascending
            e0,e1 = e1,e0; m0,m1 = m1,m0; th0,th1 = th1,th0

        th_start = lerp(e0,th0,e1,th1,e_start)
        th_end   = lerp(e0,th0,e1,th1,e_end)
        m_start  = lerp(e0,m0, e1,m1, e_start)
        m_end    = lerp(e0,m0, e1,m1, e_end)

        dtheta = max(0.0, th_start - th_end)
        bin_time_s = 2.0 * dtheta / n  # rise+set

        usable_fraction = 1.0
        if (m_start < margin_min) and (m_end < margin_min):
            usable_fraction = 0.0
        elif (m_start < margin_min) or (m_end < margin_min):
            if m_end != m_start:
                e_cross = e_start + (margin_min - m_start) * (e_end - e_start) / (m_end - m_start)
                usable_fraction = (e_end - e_cross) / (e_end - e_start) if e_start <= e_cross <= e_end else (1.0 if m_end >= margin_min else 0.0)
            else:
                usable_fraction = 0.0 if m_start < margin_min else 1.0

        data_bits = bps * bin_time_s * usable_fraction
        data_mib  = data_bits / (1024*1024*8)

        rows.append({
            "alt_km": float(alt_km),
            "elev_start_deg": float(e_start),
            "elev_end_deg": float(e_end),
            "theta_start_deg": math.degrees(th_start),
            "theta_end_deg": math.degrees(th_end),
            "bin_time_s": bin_time_s,
            "margin_start_db": m_start,
            "margin_end_db": m_end,
            "usable_fraction": usable_fraction,
            "data_bits": data_bits,
            "data_MiB": data_mib,
            "rate_bps": bps,
            "rate_unit": unit_str,
        })

    out = pd.DataFrame(rows)
    out.to_csv(out_csv, index=False)
    print(f"[{int(alt_km)} km] time≥thr: {out['bin_time_s'].sum():.1f}s  data: {out['data_MiB'].sum():.1f} MiB")
    return out

def main():
    parser = argparse.ArgumentParser(description="Create data budget for a pass from two CSVs.")
    parser.add_argument("--link-csv", required=False, help="Path to Endurosat_X-band_link_margin_vs_elevation.csv")
    parser.add_argument("--info-csv", required=False, help="Path to x_down_info.csv")
    parser.add_argument("--alt-km", type=float, required=False, help="Orbit altitude to use")
    parser.add_argument("--emin", type=float, default=None, help="Minimum elevation (deg)")
    parser.add_argument("--margin-min", type=float, default=0.0, help="Minimum link margin counted (dB)")
    parser.add_argument("--out-csv", required=False, help="Output CSV path")
    args, _ = parser.parse_known_args()

    # If no args given, auto mode (nice for notebooks)
    if len(sys.argv) == 1 or (args.link_csv is None and args.info_csv is None):
        link_csv = find_file("Endurosat_X-band_link_margin_vs_elevation.csv")
        info_csv = find_file("x_down_info.csv")
        if not link_csv or not info_csv:
            raise SystemExit("Auto-mode couldn't find the CSVs. Put them next to the script or pass --link-csv/--info-csv.")
        df = pd.read_csv(link_csv)
        alts = sorted(set(df["orbit_altitude_km"]))
        for alt in alts:
            out_csv = f"pass_data_budget_{int(alt)}km.csv"
            build_budget(link_csv, info_csv, float(alt), args.emin, args.margin_min, out_csv)
        return

    # CLI mode
    if not all([args.link_csv, args.info_csv, args.alt_km, args.out_csv]):
        parser.error("the following arguments are required: --link-csv, --info-csv, --alt-km, --out-csv")

    build_budget(args.link_csv, args.info_csv, args.alt_km, args.emin, args.margin_min, args.out_csv)

if __name__ == "__main__":
    main()


[500 km] time≥thr: 396.9s  data: 3725.6 MiB
[600 km] time≥thr: 422.5s  data: 3181.9 MiB


In [6]:
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# --- Config ---
TARGET_DURATION_S = 600.0   # 10 minutes
OUTDIR = Path(".")
FILES = None                # or like ["pass_data_budget_500km.csv", ...]
# ---------------

def find_pass_files():
    pats = [Path(".").glob("pass_data_budget_*km.csv"),
            Path("/mnt/data").glob("pass_data_budget_*km.csv")]
    files = []
    for g in pats:
        for p in g:
            if p.is_file() and p.suffix.lower()==".csv" and "pass_data_budget_" in p.name:
                files.append(p.resolve())
    seen, uniq = set(), []
    for p in files:
        if p not in seen:
            uniq.append(p); seen.add(p)
    return uniq

def scale_one(path: Path, duration_s: float):
    df = pd.read_csv(path)
    required = {"bin_time_s", "rate_bps", "usable_fraction", "alt_km"}
    missing = required - set(df.columns)
    if missing:
        print(f"⚠️  Skipping {path.name}: missing {missing}")
        return None, None

    base_time = float(df["bin_time_s"].sum())
    if base_time <= 0:
        print(f"⚠️  Skipping {path.name}: total bin_time_s is zero")
        return None, None

    scale = duration_s / base_time
    d = df.copy()
    d["bin_time_s_scaled"] = d["bin_time_s"] * scale
    d["data_bits_scaled"]  = d["rate_bps"] * d["bin_time_s_scaled"] * d["usable_fraction"]

    # Megabytes (decimal) and Mebibytes (binary)
    d["data_MB_scaled"]  = d["data_bits_scaled"] / (8 * 1e6)
    d["data_MiB_scaled"] = d["data_bits_scaled"] / (8 * 1024**2)

    # Cumulative for plotting
    d["cum_time_s_scaled"] = d["bin_time_s_scaled"].cumsum()
    d["cum_MB_scaled"]     = d["data_MB_scaled"].cumsum()

    alt = int(round(d["alt_km"].iloc[0]))
    summary = {
        "file": path.name,
        "alt_km": alt,
        "base_time_s": base_time,
        "scaled_time_s": float(d["bin_time_s_scaled"].sum()),
        "scaled_MB": float(d["data_MB_scaled"].sum()),
        "scale_factor": scale,
    }
    return d, summary

paths = [Path(p) for p in FILES] if FILES else find_pass_files()
if not paths:
    raise SystemExit("No pass_data_budget_*km.csv files found.")

OUTDIR.mkdir(parents=True, exist_ok=True)

scaled_list, summaries = [], []
for p in paths:
    d, s = scale_one(p, TARGET_DURATION_S)
    if d is None: 
        continue
    minutes = int(round(TARGET_DURATION_S/60))
    alt = s["alt_km"]
    out_csv = OUTDIR / f"pass_data_budget_{alt}km_scaled_{minutes}min.csv"
    d.to_csv(out_csv, index=False)
    scaled_list.append(d)
    summaries.append(s)
    print(f"[{alt} km] base {s['base_time_s']:.1f}s → scaled {s['scaled_time_s']:.1f}s | "
          f"data {s['scaled_MB']:.1f} MB → {out_csv.name}")

# Combined cumulative MB vs time
if scaled_list:
    plt.figure(figsize=(8,6))
    for d in scaled_list:
        alt = int(round(d["alt_km"].iloc[0]))
        plt.plot(d["cum_time_s_scaled"], d["cum_MB_scaled"], marker="o", label=f"{alt} km")
    plt.title(f"Cumulative Data vs Time (Scaled to {int(round(TARGET_DURATION_S/60))}-minute Pass)")
    plt.xlabel("Time (s)")
    plt.ylabel("Cumulative Data (MB)")
    plt.grid(True)
    plt.legend()
    png_path = OUTDIR / f"pass_data_budget_scaled_{int(round(TARGET_DURATION_S/60))}min_cumulative.png"
    plt.savefig(png_path, bbox_inches="tight")
    plt.close()
    print(f"Wrote cumulative plot → {png_path}")

pd.DataFrame(summaries)


[500 km] base 396.9s → scaled 600.0s | data 5905.5 MB → pass_data_budget_500km_scaled_10min.csv
[600 km] base 422.5s → scaled 600.0s | data 4738.2 MB → pass_data_budget_600km_scaled_10min.csv
Wrote cumulative plot → pass_data_budget_scaled_10min_cumulative.png


,file,alt_km,base_time_s,scaled_time_s,scaled_MB,scale_factor
0,pass_data_budget_500km.csv,500,396.905248,600.0,5905.492015,1.511696
1,pass_data_budget_600km.csv,600,422.499347,600.0,4738.196618,1.420121
